<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ema_supertrend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install --upgrade pandas-datareader
!pip install yfinance
!pip install numpy
!pip install matplotlib
!pip install pandas-ta

In [21]:
from dateutil.relativedelta import relativedelta
from datetime import date
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import math as math

In [ ]:
startdate = dt.today() - relativedelta(years=4)
enddate = dt.today()

df = yf.download('HINDZINC.NS' , start= startdate, end= enddate)

#Stochastic
df['14-high'] = df['High'].rolling(14).max()
df['14-low'] = df['Low'].rolling(14).min()
df['%K'] = (df['Close'] - df['14-low'])*100 / (df['14-high'] - df['14-low'])
df['%D'] = df['%K'].rolling(3).mean()

#ema
df.ta.ema(close='Close', length=200, append=True)

#Stochastic
df.ta.supertrend(high='High', low='Low', close='Close', length=10, multiplier=1, append=True)
df.ta.supertrend(high='High', low='Low', close='Close', length=11, multiplier=2, append=True)
df.ta.supertrend(high='High', low='Low', close='Close', length=12, multiplier=3, append=True)

In [85]:
def backtest_super5(signal, investment):

  close = signal['Close']
  entry = []
  exit = []
  share = 0
  price_pos = 0
  p_l = 0
  b_l = 0
  in_position = False
  equity = investment

  
  for i in range (0, len(signal)):

    
    if signal['EMA_200'][i] < close[i]:      
      if not in_position and signal['Supertrend'][i] == 3 and signal['%K'][i] < signal['%D'][i]:
        share = math.floor(equity / close[i] )
        b_l = share * close[i]
        equity -= share * close[i]
        entry.append((i, close[i]))
        price_pos = i
        in_position = True
        print(f'Buy {share} shares at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
      elif in_position and signal['High'][i] > close[price_pos]*1.03:
        p_l = share * close[price_pos]*1.03 - b_l
        equity += share * close[price_pos]*1.03
        exit.append((i, close[price_pos]*1.03))
        in_position = False
        print(f'Sell at {round(close[price_pos]*1.03,2)} on {df.index[i].strftime("%Y/%m/%d")}')
        print(f'You made {round(p_l,2)} \n')
        print(f'The Balance is {round(equity,2)} \n')
      elif in_position and (df.index[i] - df.index[price_pos]).days > 60:
        p_l = share * close[i] - b_l
        equity += share * close[i]
        exit.append((i, close[i]))
        in_position = False
        print(f'Sell at {round(close[i],2)} on {df.index[i].strftime("%Y/%m/%d")}')
        print(f'You made {round(p_l,2)} due to overextension \n')
        print(f'The Balance is {round(equity,2)} \n')
        
  if in_position:
      equity += share * close[i]
        
  earning = equity - investment
  roi = round(earning/investment*100,2)
  print(f'Earning from investing INR 100k is INR {earning} (ROI = {roi}%)')
  return entry, exit, equity


In [86]:
#df=(df.loc['2022-1-1':'2023-6-29'])
entry, exit, roi = backtest_super5(df, 100000)

Buy 303 shares at 329.7 on 2022/01/04
Sell at 339.59 on 2022/03/02
You made 2996.97 

The Balance is 102996.97 

Buy 292 shares at 351.75 on 2022/03/02
Sell at 362.3 on 2022/03/03
You made 3081.33 

The Balance is 106078.3 

Buy 332 shares at 319.5 on 2022/04/01
Sell at 329.08 on 2022/04/05
You made 3182.22 

The Balance is 109260.52 

Buy 333 shares at 327.4 on 2022/04/05
Sell at 337.22 on 2022/04/07
You made 3270.73 

The Balance is 112531.25 

Buy 336 shares at 334.85 on 2022/04/07
Sell at 344.9 on 2022/04/11
You made 3375.29 

The Balance is 115906.54 

Buy 332 shares at 348.5 on 2022/04/11
Sell at 269.05 on 2022/07/11
You made -26377.4 due to overextension 

The Balance is 89529.13 

Buy 331 shares at 270.25 on 2022/07/11
Sell at 278.36 on 2022/07/14
You made 2683.58 

The Balance is 92212.72 

Buy 324 shares at 283.95 on 2022/07/14
Sell at 292.47 on 2022/07/18
You made 2759.99 

The Balance is 94972.71 

Buy 323 shares at 293.6 on 2022/07/18
Sell at 283.8 on 2022/09/23
You made -